In [1]:
from rosetta import *
rosetta.init('-ignore_unrecognized_res T')
from sjb_scoreutil import dataframe_from_pose_and_sfxn

In [2]:
p = pose_from_pdb('/home/steve/software/BallContact_2015/test_ligand/2jie.pdb')

In [4]:
## phi angle ~ -60, plot a hist of all phi angles in prol in structure

# 1- Score WT Protein
# 2- for each position:
#        store phi,psi angles
#        mutate to proline
#        compare phi,psi angles
#        if pp < cutoff:
#           find_neighbors
#           repack/design neighbors
#       print scores
#       dump pdb

PDB file name: /home/steve/software/BallContact_2015/test_ligand/2jie.pdb
Total residues:445
Sequence: NTFIFPATFMWGTSTSSYQIEGGTDEGGRTPSIWDTFCQIPGKVIGGDCGDVACDHFHHFKEDVQLMKQLGFLHYRFSVAWPRIMPAAGIINEEGLLFYEHLLDEIELAGLIPMLTLYHWDLPQWIEDEGGWTQRETIQHFKTYASVIMDRFGERINWWNTINEPYCASILGYGTGEHAPGHENWREAFTAAHHILMCHGIASNLHKEKGLTGKIGITLNMEHVDAASERPEDVAAAIRRDGFINRWFAEPLFNGKYPEDMVEWYGTYLNGLDFVQPGDMELIQQPGDFLGINYYTRSIIRSTNDASLLQVEQVHMEEPVTDMGWEIHPESFYKLLTRIEKDFSKGLPILITENGAAMRDELVNGQIEDTGRHGYIEEHLKACHRFIEEGGQLKGYFVWSFLDNFEWAWGYSKRFGIVHINYETQERTPKQSALWFKQMMAKNGF
Fold tree:
FOLD_TREE  EDGE 1 445 -1 


In [3]:
sfxn = get_fa_scorefxn()
sfxn(p)

738.918131161304

In [4]:
fa_attr = core.scoring.ScoreType.fa_atr
fa = core.scoring.ScoreTypeManager.score_type_from_name('fa_atr')

#print res1energies.get(fa)

print sfxn.get_nonzero_weighted_scoretypes()
class PyRosettaScore(object):
    def __init__(self,sfxn):
        for i in sfxn.get_nonzero_weighted_scoretypes():
        #for i in args:
            setattr(self,str(i),core.scoring.ScoreTypeManager.score_type_from_name('%s' %i ))  
            
scoretypes = PyRosettaScore(sfxn)
print scoretypes

[fa_atr, fa_rep, fa_sol, fa_intra_rep, fa_elec, pro_close, hbond_sr_bb, hbond_lr_bb, hbond_bb_sc, hbond_sc, dslf_fa13, rama, omega, fa_dun, p_aa_pp, ref, ]


In [5]:
import pandas as pd

In [6]:
pro_mut_score_df_list = list()

pack_radius = 10

aa_bool = rosetta.utility.vector1_bool()
mutant_aa = aa_from_oneletter_code('P')
for i in range(1,21):
    aa_bool.append(i==mutant_aa) ## Clever method, credit to Evan Baugh


pym = PyMOLMover()
pym.keep_history(True)
pym.update_energy(True)

# make mutations over whole protein
#for i in range(1,p.total_residue()+1):
for i in range(1,11):

    print "Mutating Residue %s " %i

    #make mutations on test pose
    test_pose = Pose()
    test_pose.assign(p)
    task = standard_packer_task(test_pose)
    task.nonconst_residue_task(i).restrict_absent_canonical_aas( aa_bool ) #sets to just Proline
    center = test_pose.residue(i).nbr_atom_xyz() # center of residue to mutate

    testmm = MoveMap()
    
   
    for j in range(1, test_pose.total_residue()+1): # loop over protein
    # if not the mutant site, or within the pack radius, set i to no repack
        if not j == i:
            repack_this_dist = center.distance_squared( test_pose.residue(j).nbr_atom_xyz() ) 
            #print j,repack_this_dist
            
            if not (repack_this_dist  > pack_radius**2): 
                #allow repack
                task.nonconst_residue_task(j).restrict_to_repacking()
                
                #allow bb movement in movemap
                testmm.set_bb(j,1) #Boolean True = the int 1
                testmm.set_chi(j,1)
                #print "Keeping resi %s" %j
                
            else:
            #print "Preventing Packing at resi %s" %i
                task.nonconst_residue_task(j).prevent_repacking()

    
#print task
    packer = PackRotamersMover(sfxn, task)
    sfxn(test_pose)
    pym.apply(test_pose)    
    print "Native pose energy for residue %s" %i
    pre_mut_resi =  p.energies().residue_total_energies(i).sum()

    packer.apply(test_pose)
    #sfxn(test_pose)
    pym.apply(test_pose)
    #print test_pose
    print "Mutation to Proline at resi %s has energies: " %i
    post_mut_resi = test_pose.energies().residue_total_energies(i).sum()

  

    #### Doesn't Correctly include the hbond terms, need to fix
    
    for scoreterm in [ a for a in dir(scoretypes) if not a.startswith('__')]:
        #print type(getattr(scoretypes,scoreterm))
        #print scoreterm
        refenergy =  p.energies().residue_total_energies(i).get( getattr(scoretypes,scoreterm) )
        testenergy = test_pose.energies().residue_total_energies(i).get( getattr(scoretypes,scoreterm) )
        diff = testenergy - refenergy
        #print "Native: %s, Pro: %s, Delta: %s" %(refenergy, testenergy, diff)
        print "%s Change: %s" %(scoreterm, diff)
        ## same same 
## print "----- %s" %p.energies().residue_total_energies(i)[getattr(scoretypes,scoreterm)]
        ## just store and use the DF to make the deltas

    minmover = MinMover()
    minmover.score_function(sfxn)
    minmover.movemap(testmm)
#    minmover.apply(test_pose)
    
    refposephi = p.phi(i)
    refposepsi = p.psi(i)
    refposeomega = p.omega(i)
    
    ### insert movemap and backbone movents here
    print "Pre-minmover score"
    print sfxn(test_pose)
    minmover.apply(test_pose)
    print "pose minmover score"
    print sfxn(test_pose)
    
    testposephi = test_pose.phi(i)
    testposepsi = test_pose.psi(i)
    testposeomega = test_pose.omega(i)
    
    
    ### Also add in the ex1 ex2 flags 
    ###
    ### Now add a mc with the repack and minmover in cycles for each mutant
    ### Recover the low energy one, use the task operation (sets repack shell)
    ## and use movemap for both the repack and the minmover
    ##
    ##
    
    print "**"*30
    print "Previous Phi: %s Psi: %s Omega: %s" %(refposephi,refposepsi,refposeomega)
    print "New Phi: %s Psi: %s Omega: %s" %(testposephi,testposepsi,testposeomega)
    print "**"*30
    
    delta_E = post_mut_resi - pre_mut_resi
    if delta_E < 0:
        print "Resi %s -> Proline has a deltaE of %s" %(i,delta_E)
    

Mutating Residue 1 
Native pose energy for residue 1
Mutation to Proline at resi 1 has energies: 
dslf_fa13 Change: 0.0
fa_atr Change: 0.299905707627
fa_dun Change: -3.49881958926
fa_elec Change: -0.297289991939
fa_intra_rep Change: 0.209014887784
fa_rep Change: 0.0944640658194
fa_sol Change: -0.22493675078
hbond_bb_sc Change: 0.0
hbond_lr_bb Change: 0.0
hbond_sc Change: 0.0
hbond_sr_bb Change: 0.0
omega Change: 0.0
p_aa_pp Change: 0.0
pro_close Change: 1.06097481028
rama Change: 0.0
ref Change: 0.722695
Pre-minmover score
734.117283696
pose minmover score
729.903455389
************************************************************
Previous Phi: 0.0 Psi: 77.5345111646 Omega: -178.980788671
New Phi: 0.0 Psi: 77.5345111646 Omega: -178.980788671
************************************************************
Resi 1 -> Proline has a deltaE of -5.75813376892
Mutating Residue 2 
Native pose energy for residue 2
Mutation to Proline at resi 2 has energies: 
dslf_fa13 Change: 0.0
fa_atr Change: 0.47

In [25]:
#print testmm
#print task

In [50]:
phis = [ p.phi(i) for i in range(1, p.total_residue() + 1)]
psis = [p.psi(i) for i in range(1, p.total_residue() + 1)]
omegas = [p.omega(i) for i in range(1, p.total_residue() + 1)]

phis = {}
psis = {}
omegas = {}
for i in range(1,p.total_residue()+1):
    phis[i] = p.phi(i)
    psis[i] = p.psi(i)
    omegas[i] = p.omega(i)

    
for i in range(1,p.total_residue()+1):
    chis = [p.chi(j + 1, i) for j in range(p.residue(i).nchi() )]
    for chi_no in range(len(chis)):
        print 'Chi ' + str(chi_no + 1) + ':', chis[chi_no]


Chi 1: -176.263166314
Chi 2: 45.6862042589
Chi 1: 177.113934121
Chi 2: 3.39999995495e-05
Chi 1: -50.8855864587
Chi 2: 75.4193179015
Chi 1: -80.8752544668
Chi 2: 65.332471062
Chi 1: -72.6073688698
Chi 2: 74.0864880668
Chi 1: -35.796380359
Chi 2: 45.3741988602
Chi 3: -37.0959845725
Chi 1: 41.3331949565
Chi 2: 3.40000003747e-05
Chi 1: -158.383484626
Chi 2: 69.443778386
Chi 1: -33.6065973493
Chi 2: -162.552647545
Chi 3: -59.9861136752
Chi 1: -67.6829477555
Chi 2: 104.381372542
Chi 1: -178.855576137
Chi 2: 3.40000002282e-05
Chi 1: -68.4016080312
Chi 2: 4.98997728636e-13
Chi 1: 64.5798106661
Chi 2: 3.39999998992e-05
Chi 1: 152.565896639
Chi 2: -1.24077346532e-13
Chi 1: 64.3547127123
Chi 2: -9.92618772256e-15
Chi 1: -171.484640478
Chi 2: 76.2338582205
Chi 3: -6.35623574566e-13
Chi 1: -63.7059676074
Chi 2: 176.922944033
Chi 3: -97.6515959923
Chi 1: 74.2582101463
Chi 2: 177.557749936
Chi 1: -82.610502809
Chi 2: -176.317713251
Chi 3: 22.317512782
Chi 1: 50.0984082486
Chi 2: 3.40000006411e-05
Chi

In [53]:
DSSP = DsspMover()
DSSP.apply(p)    # populates the pose's Pose.secstruct                                                                       
ss = p.secstruct()
print 'Secondary Structure:\n' + ss

Secondary Structure:
LLLELLLLLEEEEELLHHHHLLLLLLLLLLLEHHHHHHHLLLLLHHHLLLLLLLLHHHHHHHHHHHHHHHLLLEEEEELLHHHHLLLLLLLLHHHHHHHHHHHHHHHHHLLEEEEEEELLLLEHHHHHLLHHHLHHHHHHHHHHHHHHHHHLLLLLLEEEEEELHHHHHHHHHHLLLLLLLLLLHHHHHHHHHHHHHHHHHHHHHHHHLLLLLEEEEEEELLLEEELLLLHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHLLLLLLLLLHHHHHLLLLLEEEEELLLLEEEEELLLLLLLLEEEELLLLLELLLLLELLHHHHHHHHHHHHHLLLLLLLEEEEEELLLLLLLLELLEELLHHHHHHHHHHHHHHHHHLLLLLLEEEEEEELLELLLLHHHHHHLLLLLEEELLLLLLEEEEHHHHHHHHHHHHLEL


In [18]:
movemaps = MoveMap()
print movemaps
movemaps.set_bb(1,1)
movemaps.set_bb(2,1)
print movemaps


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    FALSE
 DEFAULT             THETA    FALSE
 DEFAULT                 D    FALSE
 DEFAULT               RB1    FALSE
 DEFAULT               RB2    FALSE
 DEFAULT               RB3    FALSE
 DEFAULT               RB4    FALSE
 DEFAULT               RB5    FALSE
 DEFAULT               RB6    FALSE



-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
     001      BB      TRUE
     002     